## CNN

In [1]:
# Add Lincoln to system path
import sys
sys.path.append("/Users/seth/development/lincoln/")

In [2]:
from torch import Tensor
import torch

import typing
from typing import List, Tuple

**Note:** Old losses and layers code.

In [3]:
import lincoln as lnc
from lincoln.layers import Dense, Conv2D, Layer
from lincoln.losses import LogLoss, LogSigmoidLoss, MeanSquaredError
from lincoln.optimizers import SGD
from lincoln.operations import ReLU, Softmax
from lincoln.activations import Sigmoid, LogSigmoid
from lincoln.network import NeuralNetwork

In [4]:
from torchvision.datasets import MNIST
mnist_trainset = MNIST(root="./explanatory/data", train=True, download=False, transform=None)
mnist_testset = MNIST(root="./explanatory/data", train=False, download=True, transform=None)

#### One hot encode labels

In [5]:
data = mnist_trainset
num_labels = len(data.train_labels)
train_labels = torch.zeros(num_labels, 10)
for i in range(num_labels):
    train_labels[i][data.train_labels[i]] = 1
train_labels

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]])

In [6]:
data = mnist_testset
num_labels = len(data.test_labels)
test_labels = torch.zeros(num_labels, 10)
for i in range(num_labels):
    test_labels[i][data.test_labels[i]] = 1
test_labels

tensor([[0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [7]:
mnist_train = mnist_trainset.train_data.type(torch.float32).unsqueeze(3) / 255.0
mnist_test = mnist_testset.test_data.type(torch.float32).unsqueeze(3) / 255.0

In [8]:
X_train = mnist_train
X_test = mnist_test
y_train = train_labels
y_test = test_labels

In [9]:
cython = True

In [10]:
def _conv_transform(input_: Tensor) -> Tensor:

    return input_.view(input_.shape[0],
                       input_.shape[1] * input_.shape[2] * input_.shape[3])

X_train_flat = _conv_transform(X_train)
X_test_flat = _conv_transform(X_test)

In [11]:
# conv_nn_ll = NeuralNetwork(
#     layers=[Dense(neurons=250, 
#                   activation=Sigmoid()),
#             Dense(neurons=50, 
#                   activation=Sigmoid()),
#             Dense(neurons=10, 
#                   activation=Sigmoid())],
#             loss = LogLoss(),
#             optimizer = SGD(0.1))

In [12]:
# import time
# start = time.time()
# conv_nn_ll.fit(X_train_flat, y_train, X_test_flat, y_test,  
#         epochs=1,
#         eval_every=1,
#         batch_size=100,
#         seed=91418);
# print("Training took", round(time.time()-start, 1), "seconds.")

In [13]:
def eval_nn(nn, X_test, y_test):
    test_preds = nn.forward(X_test)
    
    test_preds_l = torch.argmax(test_preds, dim=1)
    
    test_vals_l = torch.argmax(y_test, dim=1)
    
    return torch.sum(test_preds_l == test_vals_l)

In [14]:
# eval_nn(conv_nn_ll, X_test_flat, y_test)

In [15]:
X_train_perm = X_train.permute(0, 3, 1, 2)
X_test_perm = X_test.permute(0, 3, 1, 2)

In [39]:
conv_nn_logloss = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=24,
                   activation=ReLU(),
                   pytorch=True),
            Conv2D(param_size=3,
                   out_channels=30,
                   activation=ReLU(),
                   pytorch=True,
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogLoss(),
            optimizer = SGD(0.1))

In [32]:
# %load_ext autoreload
# %autoreload 2

In [33]:
X_train_perm.requires_grad = True
y_train.requires_grad = True
X_test_perm.requires_grad = True
y_test.requires_grad = True

In [ ]:
import time
start = time.time()
conv_nn_logloss.fit(X_train_perm, y_train, X_test_perm, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=100,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")

322.0651550292969
322.9977111816406
331.97906494140625
321.9979248046875
338.0932922363281
327.8659362792969
335.0276794433594
327.6075439453125
309.4156494140625
327.34228515625
314.3735656738281
324.8614196777344
309.9920349121094
316.54376220703125
308.6775817871094
309.8320617675781
300.82952880859375
308.5901794433594
308.6893615722656
325.7525939941406
306.55596923828125
307.4443359375
316.1963195800781
303.03094482421875
304.9211120605469
293.7526550292969
300.8333740234375
300.2974853515625
296.581298828125
296.8754577636719
293.6629638671875
298.064208984375
307.75177001953125
296.323486328125
293.2658386230469
291.41876220703125
289.95050048828125
297.72930908203125
285.41705322265625
285.94940185546875
289.4371032714844
289.85186767578125
295.3537292480469
299.08087158203125
286.6126403808594
297.00506591796875
280.3492736816406
289.7687683105469
291.3680114746094
294.56494140625
301.017578125
294.62579345703125
283.7514343261719
286.19122314453125
290.8516540527344
282.0351

In [36]:
eval_nn(conv_nn_logloss, X_test_perm, y_test)

tensor(7553)